In [10]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import mojimoji
import re
import jaconv
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import warnings
from sklearn import preprocessing
import MeCab
def parse_text(text, debug=False):
    text = re.sub(r'［[^］]+］', ' ', text)  
#     text = re.sub(r'（[^）]+）', ' ', text)  
    text = re.sub(r'○', ' ', text)    
    text = re.sub(r'×', '', text)
    text = re.sub(r'※', '', text)    
#     text = re.sub(r'｜', ' ', text)
    text = re.sub(r'[\s、]', '' , text)
#     text = re.sub(r'一', ' ', text)
#     text = re.sub(r'…', ' ', text)  
#     text = re.sub(r'―', ' ', text)
    text = re.sub(r'[0-9]', '0', text)
    return text

def tokenize1(text):
    available_norm = ['接尾', '一般', '形容動詞語幹', 'サ変接続']
    node = mecab.parseToNode(text)
    l = []
    while node:
        l.append(node.surface)
        node = node.next
    return l
traindf=pd.read_csv("data/train.csv")
testdf=pd.read_csv("data/test.csv")
df=pd.concat([traindf,testdf])
df.body=df.body.map(parse_text)
traindf.body=traindf.body.map(parse_text)
testdf.body=testdf.body.map(parse_text)
y = df['author'].values
mecab = MeCab.Tagger('-Owakati')
count = CountVectorizer(analyzer=tokenize1)
bags = count.fit_transform(df.body.values)

features = count.get_feature_names()

bodyvec = pd.DataFrame(bags.toarray(), columns=features)
wordbody=bodyvec.sum().sort_values(ascending=False)
bodyvec1 = bodyvec[list(wordbody[wordbody > 2].index)]
xsum = bodyvec1.sum(axis=1)

xsum = np.array(xsum).reshape(len(xsum), 1)
newdf = pd.concat([df.reset_index(drop=True), pd.DataFrame(bodyvec1)/xsum*100], axis=1)
train=newdf.dropna().drop(["writing_id","body",],axis=1)
test=newdf[newdf.author.isnull()].drop(["writing_id","body"],axis=1)
test=test.drop(["author"],axis=1)
X = train.drop(["author"],axis=1)
y = train.author

mmscaler = preprocessing.MinMaxScaler()

mmscaler.fit(pd.concat([X,test]))
X = mmscaler.transform(X)
test= mmscaler.transform(test)


In [28]:
bodyvec1 = bodyvec[list(wordbody[wordbody > 10].index)]
xsum = bodyvec1.sum(axis=1)

xsum = np.array(xsum).reshape(len(xsum), 1)
newdf = pd.concat([df.reset_index(drop=True), pd.DataFrame(bodyvec1)/xsum*100], axis=1)
train=newdf.dropna().drop(["writing_id","body",],axis=1)
test=newdf[newdf.author.isnull()].drop(["writing_id","body"],axis=1)
test=test.drop(["author"],axis=1)
X = train.drop(["author"],axis=1)
y = train.author

mmscaler = preprocessing.MinMaxScaler()

mmscaler.fit(pd.concat([X,test]))
X = mmscaler.transform(X)
test= mmscaler.transform(test)

In [42]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 0) # 80%のデータを学習データに、20%を検証データにする
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
mlpr=MLPRegressor(**{"hidden_layer_sizes":(128,128,128),"random_state":42})
mlpr.fit(train_X,train_y)
pred = np.where(mlpr.predict(test_X)>0.38,1,0)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[918   1]
 [  0  75]]
accuracy =  0.9989939637826962
precision =  0.9868421052631579
recall =  1.0
f1 score =  0.9933774834437086


In [ ]:
# pred = np.where(mlpr.predict(test_X)>0.40,1,0)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

In [33]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
mlpr=MLPRegressor(**{"hidden_layer_sizes":(128,128,128),"random_state":42})
mlpr.fit(X,y)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(128, 128, 128), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=42, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [41]:
pred = np.where(mlpr.predict(test)>0.485,1,0)
# pred=mlpr.predict(test)
sub = pd.DataFrame(pd.read_csv("data/test.csv")['writing_id'])
sub["author"] = list(pred)
sub.to_csv("submission.csv", index = False)

In [35]:
pred=mlpr.predict(test)

In [36]:
target=pd.concat([y,pd.Series(pred)])

In [38]:
target=pd.concat([y,pd.Series(pred)])
data=pd.concat([pd.concat([pd.DataFrame(X),pd.DataFrame(test)]),pd.concat([y,pd.Series(pred)])],axis=1)
data.columns=list(wordbody[wordbody>10].index)+["target"]
data

,の,。,に,は,た,て,を,が,で,と,...,爆風,敷設,掛け値,伊賀山,法華経,申し入れ,法螺貝,リビヤ,濫作,target
0,0.483053,0.333931,0.584534,0.366569,0.167106,0.194406,0.489776,0.210554,0.451187,0.097625,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.442331,0.222386,0.544990,0.478479,0.264863,0.241673,0.380535,0.378598,0.308487,0.288233,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.403056,0.247671,0.227608,0.333051,0.520548,0.672877,0.466182,0.195205,0.390411,0.168950,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.404196,0.374749,0.463217,0.429419,0.608000,0.285789,0.544500,0.286036,0.116073,0.174909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
4,0.507254,0.447174,0.559759,0.484524,0.707711,0.423900,0.409564,0.296157,0.190676,0.250178,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,0.477641,0.227526,0.435189,0.315556,0.373235,0.331688,0.387431,0.356906,0.286924,0.370984,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003234
1416,0.315617,0.241694,0.320411,0.425955,0.349059,0.225229,0.337633,0.234054,0.134191,0.205949,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001530
1417,0.138380,0.239153,0.468864,0.588063,0.268078,0.129947,0.261905,0.180952,0.301587,0.391534,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050992
1418,0.203850,0.248683,0.282556,0.317329,0.348449,0.184261,0.222824,0.136846,0.123541,0.224130,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004519


In [39]:
data[(data.target<0.7)&(data.target>0.29)].target
#85 0.46 0
#277 0.40 0
#291 0.41 0
#547 0.69 1
#625 0.48 1 ○
#641 0.39 0


85      0.469008
277     0.402752
291     0.416134
547     0.693156
625     0.489638
641     0.397396
682     0.293603
702     0.687956
774     0.487221
815     0.419158
960     0.482316
962     0.315974
1050    0.334102
1082    0.433565
1106    0.542783
1112    0.459203
1116    0.538520
1140    0.341377
1200    0.504582
1344    0.434295
Name: target, dtype: float64

In [40]:
testdf.ix[data[(data.target<0.7)&(data.target>0.29)].index]

,writing_id,body
85,277,虫の中でも人間に評判のよくないものの随一《ずいいち》は蛆《うじ》である。「蛆虫めら」というの...
277,948,どうもみんな佳《よ》い言葉を使い過ぎます。美辞を姦するおもむきがあります。鴎外《おうがい》が...
291,998,一私が生れて始めて蓄音器と云うものを見聞いたのはもう十四五年前父が英国から土産に買って来たも...
547,1862,或る一つの作品を書かうと思つてそれが色々の径路を辿《たど》つてから出来上がる場合と直ぐ初めの...
625,2137,菊池は生き方が何時も徹底している。中途半端のところにこだわっていない。彼自身の正しいと思うと...
641,2170,Ｙ子はＫ病院で扁桃腺の手術をすることになつた。Ｙ子は九歳で春夏秋冬風邪をひいた。Ｙ子は母親と...
682,2321,十年前は阿佐ヶ谷に住んで居りやはり目下と同様吶々と小説ばかり書いて居りました。もう十年も経つ...
702,2385,すべて背景を用いない。宦官《かんがん》が二人話しながら出て来る。――今月も生み月になっている...
774,2649,ナポレオンの遺書――セント・ヘレナの島で臨終より三週間ほど前に彼が自ら口述し浄書したもので現...
815,2791,僕はこれからも今月のと同じような材料を使って創作するつもりである。あれを単なる歴史小説の仲間...


In [49]:
data[(data.target<0.7)&(data.target>0.29)].target
#291 0 0.31
#412 1 0.61
#547 1 0.39 ✖︎
#625 1 0.34 ✖︎
#682 0 0.54 ✖︎
#702 1 0.56 
#774 0 0.37 
#815 1 0.35 ✖︎
#825 0 0.32 
#955 1 0.59
#962 0 0.29
#1029 1 0.42 ✖︎
#1050 0 0.29
#1116 1 0.60
#1140 1 0.31 ✖︎
#1194 1 0.65
#1200 0 0.42 
#1281 0 0.44 

291     0.311938
412     0.615388
547     0.394943
625     0.348613
682     0.540990
702     0.567220
774     0.376710
815     0.357455
825     0.321671
955     0.598116
962     0.290931
1029    0.428522
1050    0.290529
1116    0.609978
1140    0.315641
1194    0.653080
1200    0.428232
1281    0.442822
Name: target, dtype: float64

In [14]:
testdf.ix[data[(data.target<0.7)&(data.target>0.29)].index]

,writing_id,body
291,998,一私が生れて始めて蓄音器と云うものを見聞いたのはもう十四五年前父が英国から土産に買って来たも...
412,1379,昔々バグダツドのマホメツト教のお寺の前に一人の乞食が寝て居りました。丁度その時説教がすんだの...
547,1862,或る一つの作品を書かうと思つてそれが色々の径路を辿《たど》つてから出来上がる場合と直ぐ初めの...
625,2137,菊池は生き方が何時も徹底している。中途半端のところにこだわっていない。彼自身の正しいと思うと...
682,2321,十年前は阿佐ヶ谷に住んで居りやはり目下と同様吶々と小説ばかり書いて居りました。もう十年も経つ...
702,2385,すべて背景を用いない。宦官《かんがん》が二人話しながら出て来る。――今月も生み月になっている...
774,2649,ナポレオンの遺書――セント・ヘレナの島で臨終より三週間ほど前に彼が自ら口述し浄書したもので現...
815,2791,僕はこれからも今月のと同じような材料を使って創作するつもりである。あれを単なる歴史小説の仲間...
825,2828,一崇拝する偉人一現存の人の裡世界第一の偉人と思惟する人一右理由一プラトーシェイキスピアゲーテ...
955,3204,一ロマンスの中の女性は善悪共皆好み候。二あゝ云ふ女性は到底この世の中にゐないからに候。


In [71]:
testdf.body[1082]

'長二《ちょうじ》は貧乏《びんぼう》の家《いえ》に生《う》まれておもちゃも持《も》たずに死《し》んでしまった。美《うつく》しいガラス張《ば》りの店頭《みせさき》に西洋《せいよう》のぜいたくな小間物《こまもの》や赤《あか》紫《むらさき》に塗《ぬ》ったゴムまりやぴかぴかと顔《かお》の映《うつ》る銀笛《ぎんてき》やらっぱやなんでも子供《こども》の好《す》きそうなものが並《なら》べてあるのを見《み》ると店《みせ》のガラス戸《ど》を砕《くだ》いてそれらのものをめちゃめちゃにたたき壊《こわ》してやりたくなる。隣《となり》に住《す》んでいたあの貧《まず》しかった哀《あわ》れな長二《ちょうじ》のことを思《おも》い出《だ》したときに。'